In [32]:
import tensorflow as tf
import matplotlib.pyplot as plt
import pandas as pd
import cv2
import numpy as np
import webbrowser

from tensorflow.keras.layers import Dense, Conv2D, MaxPooling2D, Flatten, GlobalAveragePooling2D, Dropout
from tensorflow.keras.preprocessing import image
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.metrics import TopKCategoricalAccuracy

from keras.models import model_from_json
from keras.preprocessing.image import load_img
from keras.preprocessing.image import img_to_array
from keras.applications.densenet import preprocess_input

In [33]:
# Load train images

if not os.path.exists(".model.h5"):
    train_datagen = image.ImageDataGenerator(rescale=1./255,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True,
        validation_split=0.2) # set validation split

    train_generator = train_datagen.flow_from_directory(
        '../input/car-brand-logos/Car_Brand_Logos/Train',
        target_size=(224, 224),
        batch_size=20,
        class_mode='categorical',
        subset='training') # set as training data

    validation_generator = train_datagen.flow_from_directory(
        '../input/car-brand-logos/Car_Brand_Logos/Train', # same directory as training data
        target_size=(224, 224),
        batch_size=20,
        class_mode='categorical',
        subset='validation',
        shuffle=False) # set as validation data

In [34]:
from keras.applications.vgg16 import VGG16

if not os.path.exists(".model.h5"): 
    # load pretrained model
    model = VGG16(weights='imagenet', include_top=False, input_shape=(224,224,3))

    # add trainable layers on top
    x = Flatten()(model.output)
    x = Dense(100, activation='relu')(x)
    x = Dense(100, activation='relu')(x)
    x = Dropout(0.05)(x)
    x = Dense(100, activation='relu')(x)
    x = Dropout(0.05)(x)
    predictions = Dense(8, activation='softmax')(x) 

    # set pre trained layers to not trainable
    full_model = Model(inputs=model.input, outputs=predictions)
    for layer in model.layers:
        layer.trainable = False
        
    # For training

    full_model.compile(loss='categorical_crossentropy',
                      optimizer=Adam(learning_rate=0.001),
                      metrics=['acc', TopKCategoricalAccuracy(k=3)])
    history = full_model.fit_generator(
        train_generator, 
        validation_data = validation_generator,
        workers=10,
        epochs=10
    )
    
    # save model
    model_json = full_model.to_json()
    with open("model.json", "w") as json_file:
        json_file.write(model_json)
    full_model.save_weights("model.h5")
else:
    json_file = open('model.json', 'r')
    loaded_model_json = json_file.read()
    json_file.close()
    loaded_model = model_from_json(loaded_model_json)
    # load weights into new model
    loaded_model.load_weights(".model.h5")
    print("Loaded model from disk")

Loaded model from disk


In [35]:
def process_image(image):
    
    image = cv2.resize(image, (224, 224), interpolation = cv2.INTER_AREA)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    return image

In [36]:
labels = ["hyundai", "lexus", "mazda", "mercedes", "opes", "skoda", "toyata", "volkswagen"]

def PredictImage(imgPath):
    global prediction
    global top3
    
    # preprocess image
    image = load_img(imgPath, target_size=(224, 224))
    image = img_to_array(image)
    image = image.reshape((1, image.shape[0], image.shape[1], image.shape[2]))
    image = preprocess_input(image)

    results = loaded_model.predict(image)
    pred = np.argmax(results)
    prediction = labels[pred]
    
    top_values_index = sorted(range(len(results[0])), key=lambda i: results[0][i])[-3:]
    top3ind = top_values_index[::-1]
    top3 = [labels[top3ind[0]], labels[top3ind[1]], labels[top3ind[2]]]
    
    return prediction, top3

In [37]:
cap = cv2.VideoCapture(0)

# Check if the webcam is opened correctly
if not cap.isOpened():
    raise IOError("Cannot open webcam")

i = 0
while True:
    ret, frame = cap.read()
    
    frame = cv2.resize(frame, None, fx=0.5, fy=0.5, interpolation=cv2.INTER_AREA)
    cv2.imshow("Brand Detector", frame)
    frame = process_image(frame)
    frame = frame.reshape(1,224,224,3)
    if i % 20 == 0:
        # predict and get three top most likely brands
        prediction = loaded_model.predict(frame)
        top_values_index = sorted(range(len(prediction[0])), key=lambda i: prediction[0][i])[-3:]
        
        # grab labels
        top3ind = top_values_index[::-1]
        top3 = [labels[top3ind[0]], labels[top3ind[1]], labels[top3ind[2]]]
        p_val = np.max(prediction)
        prediction = labels[np.argmax(prediction)]
        if p_val >= 1.0:
            webbrowser.open("http://"+str(prediction)+ ".com")  # Go to example.com
            break
            
    c = cv2.waitKey(1)
    i += 1
    
    if c == 27:
        break
        
cap.release()
cv2.destroyAllWindows()

Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
